In [1]:
from mitmproxy import http, ctx
import time, re
import logging
import os
import sqlite3

In [44]:
distr = str(input("Directory: ")) #Директория для файлов
start_file = distr + str(input("File to save filtred stream to: ")) #Файл, в который пойдёт весь отфильтрованый поток
data_file = distr + str(input("Final file: ")) #Файл, куда будут сохранены нужные, раскодированные данные
script = "mitmdump_script.py" #Скрипт, при помощи которого достаём необходимые данные
database = distr + str(input("Database filename: ")) #Файл с базаой данный sqlite
with open ("save.txt", "w") as f: #Файл для передачи названия конечного файла скрипту
    f.write(data_file)

Directory: /mnt/c/test/
File in wether flow goes: flow1.txt
Final file: data1.txt
Database filename: database.db


In [34]:
def StartMITMDump(file):
    os.system("mitmdump -p 8890 -w " + file + " '~t video/webm' & '~t video/mp4'")
def StopMITMDump():
    time.sleep(int(input("Time to wait before stopping the flow: ")))
    os.system("pkill mitmdump")
def ReceiveData(script, file):
    os.system("mitmdump -s " + script + " -nr " + file)

In [35]:
StartMITMDump(start_file) #Запускаем mitmdump (не включать mitmproxy, не использовать прокси сервер для локалки). 
# Собираем потоки в файл.

In [36]:
StopMITMDump() #Указываем через какое время в секундах остановить процесс mitmdump. Можно взять время с запасом
# (так как ещё есть реклама) и заниматься другими делами, предварительно отключив автовоспроизведение на ютубе.

Time to wait for stopping flow: 600


In [37]:
ReceiveData(script, start_file) #Записываем нужные данные в файл при помощи скрипта.

In [62]:
d = {}
mapp = {"133":"240 px", "134":"360 px", "135": "480 px", "136": "720 px", "137": "1080 px", "160": "144 px", 
       "264": "1440 px", "298": "720 px", "299": "1080 px", "167": "360 px", "168": "480 px", "169": "720 px",
       "170" : "1080 px", "218": "480 px", "219": "144 px", "242": "240 px", "243": "360 px", "244": "480 px", 
       "245": "480 px", "246": "480 px", "247": "720 px", "248": "1080 px", "271": "1440 px", "278": "144 px"} 
#Нужно убедиться в правильности этого, мог запутаться.
with open(data_file, "r") as f:
    a = [line.replace("\n","").split(",") for line in f.readlines()]
for idx, val in enumerate(a):
    d[idx] = {"mime":val[0], "aitag":mapp[val[1]], "duration":val[2], "range":val[3], "buffer":val[4], 
             "first request byte":val[5], "request complete":val[6], "first response byte":val[7],
             "response complete":val[8]}
d #Словарь для удобства

{0: {'mime': 'video/webm',
  'aitag': '480 px',
  'duration': '629.262',
  'range': '0-212270',
  'buffer': '0',
  'first request byte': '1542034133.744458',
  'request complete': '1542034133.7997978',
  'first response byte': '1542034133.9145322',
  'response complete': '1542034133.998709'},
 1: {'mime': 'video/webm',
  'aitag': '360 px',
  'duration': '20.000',
  'range': '0-88371',
  'buffer': '0',
  'first request byte': '1542034134.38847',
  'request complete': '1542034134.6015775',
  'first response byte': '1542034134.7556686',
  'response complete': '1542034134.8445508'},
 2: {'mime': 'video/webm',
  'aitag': '360 px',
  'duration': '20.000',
  'range': '88372-176458',
  'buffer': '0',
  'first request byte': '1542034134.939203',
  'request complete': '1542034135.1117048',
  'first response byte': '1542034135.232399',
  'response complete': '1542034135.2726324'},
 3: {'mime': 'video/webm',
  'aitag': '480 px',
  'duration': '60.000',
  'range': '0-168304',
  'buffer': '0',
  'fi

In [63]:
conn = sqlite3.connect(database) 
cursor = conn.cursor()

for row in a:
    row[1] = mapp[row[1]]
db_rows = [tuple(val) for val in a]
    
try:
    cursor.execute("""CREATE TABLE request_data
                  (mime text, aitag text, duration text,
                   range text, buffer text, first_request_byte text, request_complete text, 
                   first_response_byte text, response_complete text) """)
except:
    pass
 
cursor.executemany("INSERT INTO request_data VALUES (?,?,?,?,?,?,?,?,?)", db_rows)
conn.commit()

cursor.execute("SELECT * FROM request_data")
for idx, row in enumerate(cursor.fetchall()):
    print(str(idx) + ".", row)
cursor.close()
conn.close()

0. ('video/webm', '480 px', '629.262', '0-212270', '0', '1542034133.744458', '1542034133.7997978', '1542034133.9145322', '1542034133.998709')
1. ('video/webm', '360 px', '20.000', '0-88371', '0', '1542034134.38847', '1542034134.6015775', '1542034134.7556686', '1542034134.8445508')
2. ('video/webm', '360 px', '20.000', '88372-176458', '0', '1542034134.939203', '1542034135.1117048', '1542034135.232399', '1542034135.2726324')
3. ('video/webm', '480 px', '60.000', '0-168304', '0', '1542034135.0773451', '1542034135.1394048', '1542034135.2436464', '1542034135.27993')
4. ('video/webm', '360 px', '20.000', '176459-360179', '4171', '1542034135.3442824', '1542034135.396574', '1542034135.4382865', '1542034135.5174243')
5. ('video/webm', '480 px', '60.000', '168305-410736', '2888', '1542034135.5337577', '1542034135.626312', '1542034135.672502', '1542034135.737266')
6. ('video/webm', '360 px', '20.000', '360180-723207', '8370', '1542034135.8882706', '1542034135.9839523', '1542034136.0137217', '1542